### gemma-2b-it Instruction Tuning
preprocess에서 전처리한 데이터를 이용해 gemma 모델을 학습시킵니다.
코드들은 [Stanford alpaca](https://github.com/tatsu-lab/stanford_alpaca)를 기반으로 작성되었습니다.
학습 환경은 아래와 같습니다.

|구분|내용|
|-|-|
|학습환경|Google Colab|
|GPU|L4(22.5GB)|
|VRAM|약 13.8GB 소요|
|dtype|bfloat16|
|Attention|flash attention2|
|Tuning|Lora(r=4, alpha=32)|
|Learning Rate|1e-4|
|Optimizer|adamw_torch_fused|

---

`train.ipynb` 파일은 아래 순서로 구성되어있습니다.  

#### <span style="color:  #F2D388">1. Pretrained 모델 설정 </span>  
[gemma-1.1-2b-it](https://huggingface.co/google/gemma-1.1-2b-ithttps://huggingface.co/google/gemma-1.1-2b-it) 모델을 사용합니다.  
GPU 환경에서 [flash attention2](https://github.com/Dao-AILab/flash-attention)를 사용해주세요. 속도나 GPU 사용면에서 장점이 있으며, 퍼포먼스의 차이도 없습니다.  
device는 `cuda`로 설정, dtype은 `torch.bfloat16`으로 설정합니다.  
(CUDA Ampere 7미만은 `torch.float16`을 사용합니다.)

#### <span style="color:  #F2D388">2. Tokenizer 정의</span>  
Gemma Tokenizer로 input_ids와 labels를 생성합니다.  

#### <span style="color:  #F2D388">3. CausalLM 모델 학습한 마스킹</span>  
Prompt들을 포맷팅한 다음 토크나이징 작업을 수행합니다.  
토크나이징 후, [CausalLM 모델](https://github.com/huggingface/transformers/blob/21339a52133ba62ddc2ba5d12c68a6c053f7a4df/src/transformers/models/gemma/modeling_gemma.py#L1057https://github.com/huggingface/transformers/blob/21339a52133ba62ddc2ba5d12c68a6c053f7a4df/src/transformers/models/gemma/modeling_gemma.py#L1057)의 학습이 될 수 있도록 마스킹 작업을 수행합니다.

#### <span style="color:  #F2D388">4. Dataset과 data_collator 정의</span>  
2번의 토크나이징한 데이터로 `Dataset`을 생성하여 `train_data_set`으로 설정합니다.
`data_collator`를 통해 Sequence의 Padding을 수행합니다.

#### <span style="color:  #F2D388">5. Lora 설정</span>  
Peft 학습을 위해 LoraConfig를 설정합니다.

#### <span style="color:  #F2D388">6. 학습</span>  
1번의 Pretrained 모델, 4번의 Dataset 및 data_collator, 5번의 LoraConfig를 토대로 학습을 수행합니다.  

#### <span style="color:  #F2D388">7. 기존 모델과 비교</span>  
기존 gemma 모델과 Instruction Tuning된 모델의 답변을 비교해봅니다.

In [1]:
!pip install --quiet\
selenium\
datasets\
accelerate\
flash-attn\
peft\
trl\
transformers\
python-dotenv\
wandb\
openai

In [1]:
import os
import sys
import json
import copy
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig
import torch
from datasets import Dataset
from google.colab import userdata, drive
# utils, prompts 커스텀 모듈 사용을 위해 구글 드라이브에 마운트합니다.
drive.mount('/content/drive')

# git clone 받은 위치로 지정합니다.
# utils, prompts 커스텀 모듈을 사용하기 위해 utils.py, prompts.py가 존재하는 디렉토리롤 지정해야합니다.
path = "/content/drive/MyDrive/instruction-tuning-with-rag-example"

sys.path.append(path)
import utils
import prompts

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### <span style="color:  #F2D388">1. Pretrained 모델 설정</span>  
[google/gemma-1.1-2b-it](https://huggingface.co/google/gemma-2b-it)  
구글 Colab Secret 기능으로 허깅페이스 토큰을 사용할 수 있습니다.([Colab Secret 가이드](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75))  
gemma 모델은 토큰이 없으면 받을 수 없습니다.  

학습시 메모리 사용량을 크게 줄일 수 있는 방법은 `batch_size` 조절을 제외하고 3가지 정도가 있습니다.  
([Huggingface - Methods and tools for efficient training on a single GPU](https://huggingface.co/docs/transformers/perf_train_gpu_onehttps://huggingface.co/docs/transformers/perf_train_gpu_one)에 더 자세한 내용이 있습니다.)
+ Quantization
+ Flash Attention
+ PEFT

#### <span style="color: #ECA75D;">Quantization </span>
`torch.bfloat16` 또는 `torch.float16` 타입으로 퍼포먼스를 비교적 떨어뜨리지 않으면서 적은 메모리를 사용할 수 있습니다.  
더 자세한 내용은 [huggingface](https://huggingface.co/docs/transformers/v4.16.2/en/performance#floating-data-types)를 참고해주세요.  

#### <span style="color: #ECA75D;">Flash Attention </span>
GPU를 사용하는 환경이면 flash attention을 사용하길 추천드립니다.  
inference시 15GB정도의 VRAM이 소요되었지만, flash attention을 사용시 5GB 정도로 VRAM 사용량이 감소하였습니다.  
flash attention은 속도와 메모리 사용량을 감소시킬 수 있지만 퍼포먼스의 큰 차이가 없습니다.  
자세한 내용은 [https://arxiv.org/pdf/2307.0869](https://arxiv.org/pdf/2307.0869)을 참조해주세요.

#### <span style="color: #ECA75D;">PEFT </span>
가장 많이 사용되는 LoRA의 경우 논문에 따르면 GPU 사용량이 1/3 가량으로 절감되었다고 합니다.  
또한 Full Fine Tuning과 비교하여 퍼포먼스의 차이도 크지 않다고 합니다(어떤 경우는 퍼포먼스가 더 좋았다고 합니다).  
자세한 내용은 [https://arxiv.org/pdf/2106.09685](https://arxiv.org/pdf/2106.09685)를 참조해주세요.

In [3]:
model_id = "google/gemma-1.1-2b-it"
dtype = torch.bfloat16
token = userdata.get('HF_TOKEN_READ') # Colab Secret에서 설정해주세요
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=dtype,
    token=token,
    attn_implementation="flash_attention_2"
)
# 토크나이저의 max_length가 매우 큰수로 지정되어있으므로, 모델의 sequence length인 8192로 세팅합니다.
tokenizer.model_max_length = model.config.max_position_embeddings

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### <span style="color:  #F2D388">2. Tokenizer 정의</span>  
프롬프트들을 받아 Embedding index를 반환하는 토크나이저 기능을 정의합니다.

In [4]:
def tokenize(strings, tokenizer):
    tokenized_list = [
        tokenizer(
            text,
            max_length=tokenizer.model_max_length,
            return_tensors="pt",
            padding="longest",
            truncation=True,
            add_special_tokens=False
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

### <span style="color:  #F2D388">3. CausalLM 모델 학습한 마스킹</span>  
데이터셋은 아래의 예시와 같이 전달됩니다.  
+ `input_ids`
```python
tensor([ 28693, 238264, 236648, 234541, 90621, 235248, 235284, 237029, 47555,
         235265, 171066, 90621, 150483, 236800, 238597, 74715, 239618, 236137,
         ...
         95917, 96564, 75500, 237433, 235248, 235284, 237936, 237699, 31087,
         96564, 83160, 237036, 149735, 179694, 235265, 108, 1])
```

+ `labels`
```python
tensor([ -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100,
         ...
         95917, 96564, 75500, 237433, 235248, 235284, 237936, 237699, 31087,
         96564, 83160, 237036, 149735, 179694, 235265, 108, 1])
```

텍스트로 디코딩하면 아래와 같은 형태입니다.  
+ `input_ids`

```plain
Below is an instruction that describes a task. Write a response that appropriately completes the request.

Instruction:
Give three tips for staying healthy.

Response:
1. Eat a balanced diet and make sure to include plenty of fruits and vegetables.
2. Exercise regularly to keep your body active and strong.
3. Get enough sleep and maintain a consistent sleep schedule.</s>
```


+ `labels`

```plain
<MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK>
...
<MASK><MASK><MASK><MASK><MASK><MASK><MASK><MASK>
1. Eat a balanced diet and make sure to include plenty of fruits and vegetables.
2. Exercise regularly to keep your body active and strong.
3. Get enough sleep and maintain a consistent sleep schedule.</s>
```
`<MASK>`는 제가 임의로 적어둔 패딩 값일뿐이고, 실제로는 토큰의 인덱스 값이 -100 값으로 채워지게 됩니다.  
-100은 Pytorch의 Loss 함수에서 `ignore_index`로 사용되는 값이므로, Loss 계산시 제외됩니다.  
따라서 `input_ids`는 완전한 Sequence로 이루어지며, `labels`는 Resonpose 앞쪽이 마스킹된 형태가 됩니다.  

`input_ids`가 데이터셋에서 완전한 Sequence로 전달되어도 실제 모델에서는 [CausalMasking작업이 포함](https://github.com/huggingface/transformers/blob/2a89673fe5b946097cd483964abe543a7868aa9b/src/transformers/models/gemma/modeling_gemma.py#L944https://github.com/huggingface/transformers/blob/2a89673fe5b946097cd483964abe543a7868aa9b/src/transformers/models/gemma/modeling_gemma.py#L944)됩니다.  

즉, [CaulsalLM모델](https://github.com/huggingface/transformers/blob/2a89673fe5b946097cd483964abe543a7868aa9b/src/transformers/models/gemma/modeling_gemma.py#L1057https://github.com/huggingface/transformers/blob/2a89673fe5b946097cd483964abe543a7868aa9b/src/transformers/models/gemma/modeling_gemma.py#L1057)로 Prediction을 한 뒤, Labels와 [CrossEntropyLoss](https://github.com/huggingface/transformers/blob/2a89673fe5b946097cd483964abe543a7868aa9b/src/transformers/models/gemma/modeling_gemma.py#L1125https://github.com/huggingface/transformers/blob/2a89673fe5b946097cd483964abe543a7868aa9b/src/transformers/models/gemma/modeling_gemma.py#L1125)를 구하게 되는 과정에서 -100으로 마스킹된 토큰들은 계산에서 제외하게 하는 것입니다.  
자세한 설명은 [CausalLM을 구현한 코드](https://github.com/huggingface/transformers/blob/21339a52133ba62ddc2ba5d12c68a6c053f7a4df/src/transformers/models/gemma/modeling_gemma.py#L1120https://github.com/huggingface/transformers/blob/21339a52133ba62ddc2ba5d12c68a6c053f7a4df/src/transformers/models/gemma/modeling_gemma.py#L1120)와 [Alpaca Issue #290](https://github.com/tatsu-lab/stanford_alpaca/issues/290)을 참조하면 이해하는데 도움이 될 것 같습니다.

In [5]:
# 파이토치에서 Loss 함수들의 ignore_index 값들은 모두 -100으로 처리되어 있습니다
# https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html
IGNORE_INDEX = -100

def preprocess(
    sources,
    examples,
    tokenizer,
):
    """ 프롬프트 데이터를 받아서 전처리"""
    examples_tokenized, sources_tokenized = [tokenize(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

#### <span style="color:  #F2D388">4. Dataset과 data_collator 정의</span>
+ 데이터셋을 로드하여 프롬프트에 포맷팅합니다.
+ 포맷팅한 시퀀스를 `tokenizer`로 인코딩 및 마스킹 처리합니다.
+ 인코딩 및 마스킹 처리가 된 데이터를 `Dataset` 객체로 정의합니다.
+ batch 데이터를 처리하기 위해 `data_collator`를 정의합니다.
+ 데이터셋과 `data_collator`를 `trainer`에 전달하기 쉽게 `dict` 객체로 처리합니다.

In [6]:
from torch.utils.data import Dataset

class SupervisedDataset(Dataset):
    """데이터셋을 불러오고 프롬프트에 포맷팅합니다. 만든 데이터셋은 Dataset객체로 생성합니다."""
    def __init__(self, tokenizer, data_path):
        super().__init__()
        instructions = utils.jload(data_path)
        sources = []
        examples = []
        for idx in range(len(instructions)):
            question = json.loads(instructions[idx])['question']
            answer = json.loads(instructions[idx])['answer']
            source = prompts.GEMMA_TRAINING_PROMPT.format(question=question, answer="")
            example = prompts.GEMMA_TRAINING_PROMPT.format(question=question, answer=answer) + tokenizer.eos_token
            sources.append(source)
            examples.append(example)

        data_dict = preprocess(sources, examples, tokenizer)
        self.input_ids = data_dict['input_ids']
        self.labels = data_dict['labels']
        self.data_dict = data_dict

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i):
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

    # similar with https://github.com/huggingface/datasets/blob/4d92856bbfda0d48d07e82bb520d9434d20fae4b/src/datasets/arrow_dataset.py#L2486
    def __repr__(self):
        return f"Dataset({{\n    features: {list(self.data_dict.keys())},\n    num_rows: {self.__len__()}\n}})"


`Dataset`은 huggingface의 Dataset 클래스를 사용하는 방법도 있습니다.  
아래 코드 역시 `train_dataset`을 생성하여 datasets.Dataset 객체로 생성합니다.  
`Dataset.from_dict` 메서드를 사용하면 별도의 클래스 설정 없이 간단하게 `Dataset`을 생성합니다.  

```python
def make_train_dataset(data_path: str):
    """데이터셋을 불러오고 프롬프트에 포맷팅합니다. 만든 데이터셋은 Dataset객체로 생성합니다."""

    instructions = utils.jload(data_path)
    sources = []
    examples = []
    for idx in range(len(instructions)):
        question = json.loads(instructions[idx])['question']
        answer = json.loads(instructions[idx])['answer']
        source = prompts.GEMMA_TRAINING_PROMPT.format(question=question, answer="")
        example = prompts.GEMMA_TRAINING_PROMPT.format(question=question, answer=answer) + tokenizer.eos_token
        sources.append(source)
        examples.append(example)

    data_dict = preprocess(sources, examples, tokenizer)
    train_dataset = Dataset.from_dict(data_dict)
    train_dataset.set_format("torch")

    return train_dataset
```

In [7]:
class DataCollatorForSupervisedDataset(object):
    """데이터 콜레이터"""

    def __init__(self, tokenizer):
        self.tokenizer=tokenizer

    def __call__(self, instances):
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids_ = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels_ = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids_,
            labels=labels_,
            attention_mask=input_ids_.ne(self.tokenizer.pad_token_id),
        )

In [8]:
def make_supervised_data_module(data_path: str, tokenizer):
    """학습에 사용할 데이터셋과 콜레이터를 정의합니다."""
    train_dataset = SupervisedDataset(data_path=data_path, tokenizer=tokenizer)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
    return dict(train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator)

In [9]:
data = make_supervised_data_module(os.path.join(path, "data/instruction.jsonl"), tokenizer = tokenizer)

In [10]:
data['train_dataset']

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 9715
})

샘플로 `input_ids`를 조회해 보겠습니다.

In [11]:
data['train_dataset'].input_ids[0].to("cuda").shape

torch.Size([225])

In [12]:
print(tokenizer.decode(data['train_dataset'].input_ids[0]))

<bos><start_of_turn>user
아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다.
요청을 적절히 완료하는 응답을 작성해주세요.
단기 세를 딱 2년 주는데 미리 얘기 안해도 되는 건가요?<end_of_turn>
<start_of_turn>model
단기 임대차 계약을 체결할 때, 임대인은 임차인에게 계약 기간과 관련된 중요한 사항을 미리 고지하는 것이 바람직합니다. 임대차보호법에 따르면, 임대인은 임차인에게 계약 조건을 명확히 설명해야 하며, 이를 어길 경우 법적 분쟁이 발생할 수 있습니다. 따라서 2년이라는 단기 임대 기간을 미리 알리지 않으면 임차인이 불이익을 받을 수 있으므로, 사전에 충분히 설명하는 것이 중요합니다.
<eos>


샘플로 `labels`를 조회해보겠습니다.

In [13]:
{t: id for t, id in zip(tokenizer.all_special_ids, tokenizer.all_special_tokens)}

{2: '<bos>',
 1: '<eos>',
 3: '<unk>',
 0: '<pad>',
 106: '<start_of_turn>',
 107: '<end_of_turn>'}

-100 값이 special_token_id로 지정되지 않기 때문에 제가 임의로 -100을 `<pad>`로 replace해봤습니다.(실제로는 토큰 인덱스가 -100으로 채워집니다)  
`input_ids`는 아래처럼 처리됩니다.  

In [14]:
convert = torch.where(data['train_dataset'].labels[0] == -100, 0, data['train_dataset'].labels[0])
print(tokenizer.decode(convert))

<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>단기 임대차 계약을 체결할 때, 임대인은 임차인에게 계약 기간과 관련된 중요한 사항을 미리 고지하는 것이 바람직합니다. 임대차보호법에 따르면, 임대인은 임차인에게 계약 조건을 명확히 설명해야 하며, 이를 어길 경우 법적 분쟁이 발생할 수 있습니다. 따라서 2년이라는 단기 임대 기간을 미리 알리지 않으면 임차인이 불이익을 받을 수 있으므로, 사전에 충분히 설명하는 것이 중요합니다.
<eos>


#### <span style="color:  #F2D388">5. Lora 설정</span>  
메모리 절감을 최대화 하기 위해 모델 내 Adapter를 추가할 수 있는 모든 Layer에 LoRA를 적용하였습니다.
+ Layer이름은 모델마다 다르니 모델의 Layer를 확인하고 추가해주세요.
+ `TrainingArguments`의 경우, 여러번의 학습을 통해 적절한 하이퍼파라미터를 찾아서 학습시켰습니다.  

아래 코드는 [huggingface의 블로그](https://huggingface.co/blog/gemma-pefthttps://huggingface.co/blog/gemma-peft)를 참조하였습니다.

파라미터에 대한 설명은 huggingface에서 확인해주세요.  
+ [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments)
+ [LoraConfig](https://huggingface.co/docs/peft/package_reference/lora#peft.LoraConfig)
+ [SFTTRainer](https://huggingface.co/docs/trl/sft_trainer#trl.SFTTrainer)  

파라미터를 수정하면서 어떻게 모델이 훈련되는지 지켜보는 것도 좋은 방법입니다.  
또한 파라미터를 수정해보면서 GPU에 어떤 영향을 주는지 보는 것도 도움이 됩니다.


In [15]:
from transformers import TrainingArguments

my_model_name = os.path.join(path, "gemma-2b-it-example-v1")

args = TrainingArguments(
    output_dir=my_model_name,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=1e-4,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    max_grad_norm=0.3,
    bf16=True,
    push_to_hub=True, # 학습하면서 huggingface hub에 업로드
    hub_token=userdata.get("HF_TOKEN"),
    report_to="wandb", # wandb 사용시
    run_name="gemma-2b-it-example-v1" # wandb에서 사용할 이름
)

In [16]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.0,
        r=4,
        bias="none",
        target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
)

#### <span style="color:  #F2D388">6. 학습</span>

wandb 사용법은 [wandb guide](https://docs.wandb.ai/guides/integrations/huggingface) 페이지를 참조해주세요.  
wandb를 사용하지 않으려면, `TrainingArguments.report_to`를 제거하면됩니다.

In [22]:
import wandb
wandb_key = userdata.get("WANDB")

wandb.login(key=wandb_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from trl import SFTTrainer

data_path = os.path.join(path, "data/instruction.jsonl")
max_seq_length = model.config.max_position_embeddings
data_module = make_supervised_data_module(data_path=data_path, tokenizer=tokenizer)

trainer = SFTTrainer(
    model=model,
    args=args,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    **data_module,
)
trainer.train()
trainer.save_model()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
wandb: Currently logged in as: sk8terbo2. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.031000
20,2.102000
30,2.074800
40,1.866300
50,1.698200
60,1.605900
70,1.494900
80,1.526500
90,1.408400
100,1.444500


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=T

#### <span style="color:  #F2D388">7. 기존 모델과 비교</span>
튜닝 전 모델과 비교해보겠습니다.

In [2]:
# google/gemma-1.1-2b-it
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    token=userdata.get("HF_TOKEN"),
    attn_implementation="flash_attention_2"
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# 학습한 파인튜닝 모델
finetuned_model = AutoModelForCausalLM.from_pretrained(
    "aiqwe/gemma-2b-it-example-v1",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from utils import generate

rag_config = {
    "api_client_id": userdata.get('NAVER_API_ID'),
    "api_client_secret": userdata.get('NAVER_API_SECRET')
    }

🚀 기존 Gemma Instruction 모델

In [5]:
query = "전세사기에 대처하는 방법"

In [8]:
# 훈련 전 모델
completion = generate(
    model=model,
    tokenizer=tokenizer,
    query=query,
    max_new_tokens=512,
    rag=True,
    rag_config=rag_config
)
# output만 출력
print(completion['outputs'])

전세사기에 대처하는 방법으로는 다음과 같은 세 가지 방법이 있다.

1. **형사고소**
2. **보증금 반환 청구 소송**
3. **채권 추심 / 강제 집행**


🚀 Instruction Tuning된 모델

In [12]:
# RAG 넣어주기
completion = generate(
    model=finetuned_model,
    tokenizer=tokenizer,
    query=query,
    max_new_tokens=512,
    rag=True,
    rag_config=rag_config
)
# output만 출력
print(".\n".join(completion['outputs'].split(".")))

전세사기를 대처하려 할 때 다양한 방식들이 존재하며, 각 단계마다 신중한 접근이 중요합니다.
 우선, 경찰이나 민원인의 자문을 통해 문제를 파악한다.
 또한, 소송 과정에서 변호사와 함께 법률적인 지원을 받는 것이 효과적입니다.
 만약 재산만 나온다고 생각했던 임대료에도 불가능하다면, 은행 및 관련 업체와 충분히 검토하면 현명한 결정을 내릴 수 있다는 점을 강조했습니다.
 마지막으로, 정부 웹사이트 ' tagesun' 같은 공식 정보 체계를 이용해서 맞춤형 안내를 얻는 것도 좋은 방법입니다.




In [14]:
# RAG 없이
completion = generate(
    model=finetuned_model,
    tokenizer=tokenizer,
    query=query,
    max_new_tokens=512,
    rag=False,
    rag_config=rag_config
)
# output만 출력
print(".\n".join(completion['outputs'].split(".")))

전세 사기를 대처하기 위해서는 몇 가지 중요한 절차를 따르는 것이 필요합니다.
 첫째, 전입신고와 확정일자를 받아 임대인의 동의 없이도 거주할 수 있는 권리를 보호받기 위한 조치가 필수적입니다.
 이를 통해 경매나 공매 시 우선 변제권을 행사하여 자산에 손실을 입지 않도록 할 수 있습니다.
 둘째, 법률 상담원과 연계해 소송이나 민사청구 등 다양한 형태로 문제를 해결하고 피해 복구를 진행해야 합니다.
 예를 들어, 부동산거래관리법상 세입자가 계약갱신요구권을 사용하지 않는 경우에도 집단소송으로 큰 금액 회수라는 기회가 있음을 알리고 있다면 신속하게 적극적으로 참여하겠다는 의지를 담은 내용증명서 발급 및 배포등 방식에서 유연성을 제공한다는 점에서 주목받고 있습니다.
 마지막으로, 정부 지원 프로그램들을 활용하면 재건축 아파트 매물 처분 관련 분쟁 발생시 도움을 줄 수 있어 안심하세요.
 따라서 전문적인 지식과 기술을 이용하며 빠른 응답을 준비했습니다.



RAG를 넣었을 때 generation이 더 안좋은것 같습니다. RAG에 들어가는 input을 확인해보겠습니다.

In [17]:
completion = generate(
    model=finetuned_model,
    tokenizer=tokenizer,
    query=query,
    max_new_tokens=512,
    rag=True,
    rag_config=rag_config
)

In [18]:
# input만 출력
print(completion['inputs'])

전세사기에 대처하는 방법
If you need more information, search information through below <documents> tag.
You should answer with Korean.
<documents>
-현직 공인중개사가 알려주는 부동산 전세사기 대처방법을 알려드립니다. 전세사기란,,,,??? 전입신고, 확정일자를 받아도 이 기존 법의 허점을 역이용하여 다시 대출을 받거나, 보증금을 그냥 꿀꺽하는 전개로... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권추심/강제집행이 있습니다. 이 세 가지 절차가 유기적으로 진행되어야 전세사기 사건에서 피해를 회복할 수 있습니다. 공인중개사가 전세사기에 적극... 
 -... 전세사기에 대처하는 방법으로는 (1) 형사고소, (2) 보증금반환청구소송, (3) 채권

RAG에 제공되는 정보들이 학습에 사용한 데이터셋보다 내용이 부실합니다.  
RAG로 제공되는 데이터가 좀더 구체적이고, 정확하고, 도움이 되는 정보라면 RAG를 추가했을 때 좋은 퍼포먼스를 기대해볼 수 있을 것 같습니다.  
사내 보안이 필요한 데이터에 접근한다면 [milvus](https://milvus.io/), [faiss](https://github.com/facebookresearch/faiss)같은 VectorDB, [OpenSearch](https://opensearch.org/)를 이용한 검색엔진을 추가하는 것도 좋은 방법이 될 것 같습니다.  
Vector DB 같은 경우 [ChromaDB](https://www.trychroma.com/)를 사용하면 높은 수준의 기능을 제공받긴 어렵지만 빠르게 활용해볼 수 있습니다.